# Training notebook

This notebook walks through the training impelmentation of the project. The Machine learning models used is:
- SVM (one-class)
- 

The deep learning models implemented include:
- 

the dataset used is the 'flattened_dataset.csv' which include the following features:
- 

In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import OneClassSVM
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer, f1_score


In [2]:
columns = ["label", "lat", "lon"]
columns.extend([f"enc_desc_{i}" for i in range(768)])
columns.extend([f"enc_rdesc_{i}" for i in range(768)])

columns.extend([f"cond_desc_{i}" for i in range(768)])
columns.extend([f"careplan_desc_{i}" for i in range(768)])
columns.extend([f"careplan_rdesc_{i}" for i in range(768)])
columns.extend([f"proc_desc_{i}" for i in range(768)])
columns.extend([f"proc_rdesc_{i}" for i in range(768)])


df = pd.read_csv("flattened_dataset.csv", index_col=0, header=None)
df.columns = columns

In [3]:
df.head()

,label,lat,lon,enc_desc_0,enc_desc_1,enc_desc_2,enc_desc_3,enc_desc_4,enc_desc_5,enc_desc_6,...,proc_rdesc_758,proc_rdesc_759,proc_rdesc_760,proc_rdesc_761,proc_rdesc_762,proc_rdesc_763,proc_rdesc_764,proc_rdesc_765,proc_rdesc_766,proc_rdesc_767
0,,,,,,,,,,,,,,,,,,,,,
094e2672-7c1b-eba8-3405-20be988a3811,0,40.873811,-109.431528,-0.031033,0.289719,0.027958,-0.227295,-0.150596,0.169259,0.281590,...,-0.330483,-0.023646,0.022632,-0.018505,0.180497,-0.034652,-0.113333,0.324499,0.288531,-0.020623
c6c8fad2-773f-4a81-7737-d4ded883f521,0,40.589988,-111.721005,0.035127,0.249736,-0.092530,-0.167370,-0.073288,0.214831,0.330113,...,-0.330803,-0.023421,0.022981,-0.019290,0.180746,-0.034797,-0.113931,0.324178,0.289040,-0.020862
6e18deb0-7724-65e6-3b62-c052d30cd63e,0,40.697847,-112.205052,-0.027772,0.287580,0.018022,-0.225007,-0.141567,0.172837,0.283242,...,-0.330576,-0.023856,0.022236,-0.018447,0.179926,-0.034598,-0.113134,0.324633,0.288424,-0.021169
3829fbc5-cfb0-341d-b251-27b25ab7cc41,0,40.620239,-111.918419,0.013951,0.262305,-0.105305,-0.211218,-0.131809,0.188591,0.304787,...,-0.325680,-0.028446,0.028082,-0.013930,0.180542,-0.037471,-0.116612,0.320529,0.285655,-0.016130
58460ba5-986e-89eb-3d53-cc3133658f46,0,40.686387,-111.985047,0.047515,0.235343,-0.095788,-0.169949,-0.086596,0.217631,0.326041,...,-0.328867,-0.025385,0.023512,-0.017824,0.180867,-0.034692,-0.114863,0.322222,0.287333,-0.019064


In [ ]:
X = df.drop('label', axis=1)  # Features
y = df['label']  # Labels

# One-class SVM

In [11]:

# Filter the data to include only the 'in-class' instances
in_class_label = 1  # Change this to the label of your in-class
X_in_class = X[y == in_class_label]

# Define the model
svm_model = OneClassSVM()

# Set up parameter grid for hyperparameter tuning
param_grid = {
    'nu': np.linspace(0.01, 0.5, 50),  # An upper bound on the fraction of margin errors
    'gamma': ['scale', 'auto'] + list(np.logspace(-9, 3, 13))
}

def anomaly_detection_scoring(y_true, y_pred):
    # Assuming your in-class data is labeled as 1 and out-class as -1 or 0
    # You might need to adjust the labels based on how your data is structured
    y_true_converted = np.where(y_true == in_class_label, 1, -1)
    return f1_score(y_true_converted, y_pred, pos_label=1)

# Make the scorer using make_scorer function
custom_scorer = make_scorer(anomaly_detection_scoring)

# RandomizedSearchCV configuration
random_search = RandomizedSearchCV(svm_model, param_grid, n_iter=2, verbose=3, cv=KFold(n_splits=5), scoring=custom_scorer, n_jobs=-1)

# Train the model
random_search.fit(X_in_class)

# Best model and parameters
best_model = random_search.best_estimator_
print("Best parameters:", random_search.best_params_)

KeyboardInterrupt: 